
# BRAS Reference Scene RS1

This Notebook reproduces the BRAS Reference Scene RS1 with Mitsuba3.

A speaker and a microphone are positioned above an absorber panel which is flat on a fully reflecting concrete floor. 


<div align='center'>
<img src='1 Scene descriptions/RS1 single reflection (infinite plate)/Geometry/RS1_RIR_Absorbing.png' alt='Alt text' style='width:70%;'>
</div>


In [ ]:
import numpy as np
from os import path
import matplotlib.pyplot as plt
import drjit as dr
from platform import system
import pyfar as pf
import sofar as sf
import mitsuba as mi

if system() == 'Darwin':
    mi.set_variant('llvm_ad_rgb')
else:
    mi.set_variant('cuda_ad_rgb')
print(f'Mitsuba variant: {mi.variant()}')

from mitsuba import ScalarTransform4f as T


# set retina backend for plotting with matplotlib magic
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

mi.set_log_level(mi.LogLevel.Warn)

# Microphone and speaker positions

The Room Impulse Responses (RIRs) are stored in the measurement SOFA-file. The SOFA-file contains the positions of the emitter and receiver as well.

In [ ]:
sofafile = sf.read_sofa('1 Scene descriptions/RS1 single reflection (infinite plate)/RIRs/RS1_RIRs_Absorbing.sofa')
# sofafile.info()

The DATA is stored in 9 channels for the 9 combinations of Emitter and Receiver positions. We can set the receiver and emitter ID and then select the corresponding channel of the Data

In [ ]:
EmitterID = 2
ReceiverID = 2

In [ ]:
print(f'Shape of the RIR Data: {sofafile.Data_IR.shape}')

# find sofafile index by comparing mic_ID and speaker_ID to sofafile.EmitterID and sofafile.ReceiverID
sofafile_index = np.where((sofafile.EmitterID == EmitterID) & (sofafile.ReceiverID == ReceiverID))[0][0]

print(f'Using sofafile index {sofafile_index} for Mic ID {EmitterID} and Speaker ID {ReceiverID}')

In [ ]:
mic_pos = sofafile.ReceiverPosition[0, :, sofafile_index]
emitter_pos = sofafile.EmitterPosition[0, :, sofafile_index]
reflection_pos = np.array([5.500, 2.985, 0.020]) # the reflection position is designed to be at the center of the absorber

# in the sofa files, z=0 is at the height of the absorber. In the Mitsuba scene, z=0 is at floor height.
# therefore, we need to shift the z coordinate by the absorber height (20 mm)
mic_pos[2] += 0.02
emitter_pos[2] += 0.02


print(f'\nEmitter ID: {EmitterID}, Receiver ID: {ReceiverID}')
print(f'Emitter position:\t{emitter_pos}')
print(f'Receiver position:\t{mic_pos}')
print(f'Reflection position:\t{reflection_pos}')

# Visual Rendering of the Scene

Configuration parameters:

In [ ]:
config_visual = {
    'mic_pos':        mic_pos,
    'speaker_pos':    emitter_pos,
    'reflection_pos': reflection_pos,

    'camera_pos': [5.5, -6, 5],
}

Dictionary that we will use to build the scene, all objects, materials and parameters are defined in here.

In [ ]:
scene_dict_visual = {
    'type': 'scene',

    'integrator': {
        'type': 'path',
        'max_depth': 3,
    },

    'sensor': {
        'type': 'perspective',
        'to_world': T.look_at(
            origin=config_visual['camera_pos'],
            target=config_visual['reflection_pos'],
            up=[0, 0, 1]
        ),
        'fov': 65,
        'film': {
            'type': 'hdrfilm',
            'width': 1920,
            'height': 1080,
        },
    },

    'speaker': {
        'type': 'cube',
        'to_world': T.look_at(
            origin=config_visual['speaker_pos'],
            target=config_visual['reflection_pos'],
            up=[1, 0, 0]
        ).translate((0, 0, -0.2)).scale((0.2, 0.3, 0.2)),
        'material': {
            'type': 'diffuse',
            'reflectance': 0.3
        }
    },

    'microphone': {
        'type': 'cylinder',
        'to_world': T.look_at(
            origin=config_visual['mic_pos'],
            target=config_visual['reflection_pos'],
            up=[0, 0, 1]
        ).translate((0, 0, -0.2)).scale((0.05, 0.05, 0.2)),
        'material': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': (0, 1, 0)
            },
        },
    },

    'microphone_sphere': {
        'type': 'sphere',
        'radius': 0.05,
        'center': config_visual['mic_pos'],
        'material': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': (0, 1, 0)
            },
        },
    },

    'concrete': {
        'type': 'rectangle',
        'to_world': T.translate(config_visual['reflection_pos']).scale((5.5, 2.985, 1)),
        'material': {
            'type': 'plastic',
            'int_ior': 1.9,
            'diffuse_reflectance': {
                'type': 'rgb',
                'value': np.array((53,33,20))/256
            }
        }
    },

    'absorber': {
        'type': 'rectangle',
        'to_world': T.translate(config_visual['reflection_pos']).translate((0, 0, 0.02)).scale((5.5-3.397, 2.985-0.881, 1)),
        'material': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': np.array((254/256, 250/256, 241/256))
            },
        },
    },

    'ceiling_light': {
        'type': 'rectangle',
        'flip_normals': True,
        'to_world': T.translate(config_visual['reflection_pos']).translate((0, 0, 10)).scale((.8, .7, 1)),
        'emitter': {
            'type': 'area',
            'radiance': {
                'type': 'rgb',
                'value': 200.0,
            }
        }
    },

    'background_floor': {
        'type': 'rectangle',
        'to_world': T.translate((0, 0, -0.001)).scale((30, 30, 1)),
        'material': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'checkerboard',
                'to_uv': T.scale([30, 30, 1])
            }
        }
    },
}

Render the visual scene.

If the rendering process takes a long time, reduce the samples per pixel `spp`. The renderer produces a noisy but still usable image even with `spp=1`.

In [ ]:
scene = mi.load_dict(scene_dict_visual)

img = mi.render(scene, spp=50)

plt.figure(figsize=(15, 12))
plt.title(f'Microphone position {EmitterID}, Speaker position {ReceiverID}')
plt.imshow(img** (1.0 / 2.2))
plt.axis('off')
plt.show()

# Load Material Parameters

The files `mat_RockFonSonarG_00deg.csv` and `mat_CR1_concrete.csv` contain data for the absorption and scattering coefficients of the concrete flooring and the absorber panels.

We want to store the data in separate lists. Also, to reduce rendering time, we only want to use the frequencies specified in `frequencies`.

<div class="alert alert-warning">
<strong>Info:</strong> This way of storing the absorption data will be deprecated once full spectral acoustic rendering is implemented.

Once implemented, the acoustic variant will work analoguously to the spectral variant and the `tape` plugin will work analoguously to the `specfilm` plugin, where the absorption and scattering spectra will need to be loaded as `acoustic` spectrum types and the tape plugin will have channels with channel-specific sensitivity spectra, e.g. octave bands or third octave bands. 
</div>

In [ ]:
path_materials = '3 Surface descriptions/_csv/initial_estimates'

path.join(path_materials, 'mat_RockFonSonarG_00deg.csv')
absorber = np.genfromtxt(path.join(path_materials, 'mat_RockFonSonarG_00deg.csv'), delimiter=',')
concrete = np.genfromtxt(path.join(path_materials, 'mat_CR1_concrete.csv'), delimiter=',')

# Reduce the size of the absorption data to the frequencies specified below
frequencies = np.array([125, 16000])
# frequencies = absorber[0] # uncomment this line to render all frequencies instead.

absorption_absorber = absorber[1, np.isin(absorber[0], frequencies)]
scattering_absorber = absorber[2, np.isin(absorber[0], frequencies)]

absorption_concrete = concrete[1, np.isin(concrete[0], frequencies)]
scattering_concrete = concrete[2, np.isin(concrete[0], frequencies)]

print(f'Frequencies:\t\t{frequencies}')
print(f'Absorption absorber:\t{absorption_absorber}')
print(f'Scattering absorber:\t{scattering_absorber}')
print(f'Absorption concrete:\t{absorption_concrete}')
print(f'Scattering concrete:\t{scattering_concrete}')

# Acoustic Rendering

First, set the variant to an acoustic variant:

In [ ]:
if system() == 'Darwin':
    mi.set_variant('llvm_ad_acoustic')
else:
    mi.set_variant('cuda_ad_acoustic')
print(f'Mitsuba variant: {mi.variant()}')

Important scene parameters like the speed of sound, absorption and scattering spectra, as well as the number of time bins and the maximum time are set here.

In [ ]:
config_acoustics = {
    'speed_of_sound': 343.0,
    'speaker_radius': 0.1,

    'absorption_concrete': [(i + 1, a) for i, a in enumerate(absorption_concrete)],
    'scattering_concrete': [(i + 1, s) for i, s in enumerate(scattering_concrete)],

    'absorption_absorber': [(i + 1, a) for i, a in enumerate(absorption_absorber)],
    'scattering_absorber': [(i + 1, s) for i, s in enumerate(scattering_absorber)],

    'integrator': 'prb_acoustic',
    'wav_bins': len(absorption_concrete),   # x
    'max_depth': 2,                         # maximum number of bounces, 2 is enough because there is only one reflection
    'max_time':  0.025,
    'time_bins': 750,                       # y
    'spp': 2**22,                           # samples per time bin

}

Dictionary that we will use to build the acoustic scene, all objects, materials and parameters are defined in here.

The acoustic scene uses an acoustic integrator, the acoustic BSDFs and the actual speaker and microphone as emitter and sensor, respectively. Otherwise, the scene definition is the same as the visual scene.

In [ ]:
scene_dict_acoustic = {
    'type': 'scene',

    'integrator': {
        'type': config_acoustics['integrator'],
        'max_depth': config_acoustics['max_depth'],
        'max_time': config_acoustics['max_time'],
        'speed_of_sound': config_acoustics['speed_of_sound'],
        'skip_direct': False,
    },

    'microphone': {
        'type': 'microphone',
        'to_world': T.translate(mic_pos),
        'film': {
            'type': 'tape',
            'wav_bins':  config_acoustics['wav_bins'],
            'time_bins': config_acoustics['time_bins'],
            'rfilter': {'type': 'box'},
            'count': True
        },
        'sampler': {'type': 'stratified', 'sample_count': config_acoustics['spp']},
    },

    'speaker': {
        'type': 'sphere',
        'radius': config_acoustics['speaker_radius'],
        'center': emitter_pos,
        'emitter': {'type': 'area', 'radiance': {'type': 'uniform', 'value': 100.}},

    },

    'absorber': {
        'type': 'rectangle',
        'to_world': T.translate(config_visual['reflection_pos']).translate((0, 0, 0.02)).scale((5.5-3.397, 2.985-0.881, 1)),
        'material': {
            'type': 'acousticbsdf',
            'absorption': {
                'type': 'spectrum',
                'value': config_acoustics['absorption_absorber'],
            },
            'scattering': {
                'type': 'spectrum',
                'value': config_acoustics['scattering_absorber'],
            },
        }
    },

    'concrete': {
        'type': 'rectangle',
        'to_world': T.translate(config_visual['reflection_pos']).scale((5.5, 2.985, 1)),
        'material': {
            'type': 'acousticbsdf',
            'absorption': {
                'type': 'spectrum',
                'value': config_acoustics['absorption_concrete'],
            },
            'scattering': {
                'type': 'spectrum',
                'value': config_acoustics['scattering_concrete'],
            },
        }
    },
}
scene = mi.load_dict(scene_dict_acoustic)

Render the scene. Note that for acoustic path tracing we need to set `spp` bin a lot higher than in the visual rendering to get good results. 

In [ ]:
data = np.array(mi.render(scene, seed=0, spp=config_acoustics['spp']))
hist = data[:, :, 0] / config_acoustics["spp"]

To make sure that our renderer produces the correct results we can calculate the time of arrival (TOA) of the direct sound and the floor reflection by hand:

In [ ]:
# Expected TOA of direct sound:
TOA_direct = np.linalg.norm(emitter_pos - mic_pos)  / config_acoustics['speed_of_sound']

# Expected TOA of reflected sound:
TOA_reflected = (np.linalg.norm(emitter_pos - reflection_pos) + np.linalg.norm(reflection_pos - mic_pos) ) / config_acoustics['speed_of_sound']

Plot the results:

In [ ]:
time = np.linspace(0., config_acoustics['max_time'], config_acoustics['time_bins'], endpoint=True)

plt.plot(time, hist, label=frequencies)
plt.ylabel('Energy')
plt.xlabel('Time in ms')
plt.vlines(TOA_direct, -np.max(hist)*0.1, 0, colors='r', linestyles='dashed', label='TOA direct')
plt.vlines(TOA_reflected, -np.max(hist)*0.1, 0, colors='g', linestyles='dashed', label='TOA reflected')
plt.ylim(-np.max(hist)*0.1, np.max(hist)*1.1)

plt.legend(title='Frequency in Hz')
plt.show()

You can see that the peaks of the direct sound and reflected sound appear just before the estimated TOAs. This is not a misaligned time axis, nor is this an artifact of unsufficient time bin resolution. 

This behavior is expected because the speaker is not modeled as a point source but as a sphere with non-zero radius. 

The estimated TOAs shown in the dashed lines are the time it takes the sound to propagate from the _center_ of the speaker to the microphone. The histogram data shows the time it took the rays to propagate from the microphone to the _surface_ of the sphere.

The difference between the point source distance and the actual ray lengths can range from 0 (the ray hits the speaker sphere tangentially) to the speaker radius (the ray hits the speaker parallel to the sphere's normal vector). The following plot shows that the histogram peaks correspond to this time range:

In [ ]:
t_correct = config_acoustics['speaker_radius']/config_acoustics['speed_of_sound']

%matplotlib inline
plt.plot(time, hist, label=frequencies)
plt.ylabel('Energy')
plt.xlabel('Time in s')
plt.vlines(TOA_direct, -np.max(hist)*0.1, 0, colors='r', linestyles='dashed', label='direct sound')
plt.vlines(TOA_direct-t_correct, -np.max(hist)*0.1, 0, colors='r', linestyles='dashed',)
plt.vlines(TOA_reflected, -np.max(hist)*0.1, 0, colors='g', linestyles='dashed', label='reflection')
plt.vlines(TOA_reflected-t_correct, -np.max(hist)*0.1, 0, colors='g', linestyles='dashed')
plt.ylim(-np.max(hist)*0.1, np.max(hist)*1.1)
plt.xlim(0, 0.025)
plt.title('TOA estimation with non-zero speaker radius')
plt.legend(title='Frequency in Hz')
plt.show()

# Comparison to measured RIR

To compare the simulation to the measured data, we can select the correct Impulse Response from the SOFA file with the `sofafile_index` we used to select the correct Emitter and Receiver positions.

In [ ]:
IRs, _, _ = pf.io.read_sofa('1 Scene descriptions/RS1 single reflection (infinite plate)/RIRs/RS1_RIRs_Absorbing.sofa')
impulse_response = IRs[sofafile_index]

print(f'\nIRs:\n{IRs}')
print(f'impulse_response:\n{impulse_response}')

We can plot the impulse response in the time domain with the `pyfar.plot.time` function:

In [ ]:
fig, ax = plt.subplots(2, 1)
pf.plot.time(impulse_response, unit='s', ax=ax[0], dB=False)
pf.plot.time(impulse_response, unit='s', ax=ax[1], dB=True)
ax[0].set_xlim(0, 0.025)
ax[1].set_xlim(0, 0.025)
plt.tight_layout()
plt.show()

We can compare the squared impulse response to the energy histogram. We can average the simulated data across the simulated frequencies to get the broadband histogram.

Apart from the time shift we introduced with the non-zero emitter radius (discussed earlier), the results of the simulation match the measurement.

In [ ]:
hist_avg = np.mean(hist, axis=1)

plt.figure()
plt.plot(impulse_response.times, impulse_response.time[0,:]**2/np.max(impulse_response.time[0,:]**2), label='Measured')
plt.plot(time, hist_avg/np.max(hist_avg), label=f'Simulated')

plt.vlines(TOA_direct, -0.05, 0, colors='r', linestyles='dashed', label='TOA direct')
plt.vlines(TOA_reflected, -0.05, 0, colors='g', linestyles='dashed', label='TOA reflected')

plt.xlim(0, 0.025)
plt.ylim(-0.05, 1.1)
plt.xlabel('Time in s')
plt.ylabel(r'$E/E_\mathrm{max}$')
plt.title('Comparison of measured and simulated energy histograms')
plt.legend()
plt.show()